In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model
from sklearn.isotonic import IsotonicRegression
from ideas import intersection_over_union
from ideas import bb_intersection_over_union
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, StratifiedKFold

In [128]:
box_columns = ['Xmin','Ymin','Xmax','Ymax']
box_columns_gd = ['Xmin_gd','Ymin_gd','Xmax_gd','Ymax_gd']
center_columns = []
# ['XcenterDelta', 'YcenterDelta']
# 'Xratio', 'Yratio'
columns_ratio = ['box_iou', 'iou']
box_columns_all = box_columns + box_columns_gd + center_columns
box_columns_true = ['Xmin_true','Ymin_true','Xmax_true','Ymax_true']
box_columns_id = box_columns + ['itemId']

In [129]:
te_d = pd.read_csv('test_data_full.csv')
t_d = pd.read_csv('train_data_full.csv')
t_a = pd.read_csv('train_answers.csv')
t_d_a = t_d.merge(t_a, on="itemId")
ids = list(set(te_d['itemId']))
scaler = MinMaxScaler()

In [130]:
scaler_fit_d = t_d_a[box_columns_all].values
scaler_fit_d = np.append(scaler_fit_d, te_d[box_columns_all].values, axis = 0)
scaler.fit(scaler_fit_d)
t_d_a[box_columns_all] = scaler.transform(t_d_a[box_columns_all])
te_d[box_columns_all] = scaler.transform(te_d[box_columns_all])

In [131]:
def my_custom_loss_func(y_true, y_pred):
    iou = np.zeros(len(y_true))
    for i in range(0, len(y_pred)):
        iou[i] = bb_intersection_over_union(y_true.iloc[i].values, y_pred[i])                          
    return iou.mean()

In [132]:
# for uid in uids:
X = t_d_a[box_columns_all + columns_ratio]
y = t_d_a[box_columns_true]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
clf = linear_model.RidgeCV(alphas=(0.1, 1, 10), 
                           cv=5, scoring=make_scorer(my_custom_loss_func))
# clf = linear_model.LassoLarsCV()
clf.fit(X_train, y_train)
y_test_pred = clf.predict(X_test)

In [133]:
iou = np.zeros(len(y_test_pred))
for i in range(0, len(y_test_pred)):
    iou[i] = bb_intersection_over_union(y_test_pred[i], y_test.iloc[i].values)
print(iou.mean())    

0.5646539873002193


In [84]:
X_submit = te_d[box_columns_all + columns_ratio]
y_submit = clf.predict(X_submit)
answers = pd.DataFrame(np.zeros((len(te_d), 5), dtype=int), index=te_d.index, columns=box_columns_id)
y = 0
for i,r in te_d.iterrows():        
    for ci,cv in enumerate(box_columns):
        answers.iloc[i][cv] = y_submit[y,ci] 
    answers.iloc[i]['itemId'] = r['itemId']
    y = y + 1

In [85]:
answers.groupby("itemId").mean().to_csv('submit2001.csv', header=False)